In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gtsa
import pandas as pd

### Lazy load co-registered DEMs from S3

In [3]:
s3_bucket_name = 'petrichor'
folder = 'baker/dems/'

In [4]:
cog_urls = gtsa.io.parse_urls_from_S3_bucket(s3_bucket_name,
                                             aws_server_url = 's3.us-west-2.amazonaws.com',
                                             folder = folder)

date_strings = gtsa.io.parse_timestamps(cog_urls,
                                        date_string_pattern='....-..-..')

date_times = [pd.to_datetime(x) for x in date_strings]

In [5]:
len(cog_urls) == len(date_times)

True

In [6]:
ref_dem = cog_urls[-1]
ref_dem

'http://petrichor.s3.us-west-2.amazonaws.com/baker/dems/dem_2015-08-27_WA_reproj_COG.tif'

In [8]:
ds = gtsa.io.xr_stack_geotifs(cog_urls,
                              date_times,
                              ref_dem,
                              resampling="bilinear",
                              save_to_nc = True,
                              nc_out_dir = '/mnt/working/knuth/sw/repos/gtsa/scripts/stacks/baker/nc_files',
                              overwrite = False)

Saved .nc files in /mnt/working/knuth/sw/repos/gtsa/scripts/stacks/baker/nc_files


In [9]:
ds

<xarray.Dataset>
Dimensions:      (x: 25279, y: 24332, time: 12)
Coordinates:
  * x            (x) float64 -122.0 -122.0 -122.0 ... -121.7 -121.7 -121.7
  * y            (y) float64 48.88 48.88 48.88 48.88 ... 48.62 48.62 48.62 48.62
  * time         (time) datetime64[ns] 1947-09-14 1970-09-09 ... 2015-08-27
    spatial_ref  int64 0
Data variables:
    band1        (time, y, x) float32 dask.array<chunksize=(1, 24332, 25279), meta=np.ndarray>
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [15]:
ds['band1'].data = ds['band1'].data.rechunk({0:'auto', 1:'auto', 2:'auto'}, 
                                                    block_size_limit=1e8, 
                                                    balance=True)

In [17]:
ds

<xarray.Dataset>
Dimensions:      (x: 25279, y: 24332, time: 12)
Coordinates:
  * x            (x) float64 -122.0 -122.0 -122.0 ... -121.7 -121.7 -121.7
  * y            (y) float64 48.88 48.88 48.88 48.88 ... 48.62 48.62 48.62 48.62
  * time         (time) datetime64[ns] 1947-09-14 1970-09-09 ... 2015-08-27
    spatial_ref  int64 0
Data variables:
    band1        (time, y, x) float32 dask.array<chunksize=(1, 4867, 5056), meta=np.ndarray>
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [ ]:
ds['band1'].sel(time = ds.time.values[0])

### Write to Zarr file

In [ ]:
ds_zarr = gtsa.io.create_zarr_stack(ds,
                                    output_directory = './zarr_stack_rainier',
                                    overwrite = False)

In [ ]:
ds_zarr

In [ ]:
ds_zarr['band1'].sel(time = ds_zarr.time.values[0])